In [1]:
import pandas as pd
import os
import requests, zipfile, io
import numpy as np


In [2]:
managers = pd.read_csv("data/ipo_managers.csv")

In [3]:
age = pd.read_csv("data/ipo_age.csv")

# Financial Data

In [4]:
# import financials -- all num.txt and sub.txt
years = list(range(2009,2021))
quarters = ['q1','q2','q3','q4']
filenames = ['num.txt','sub.txt']   
dataset = 'financial-statement-data-sets'
# 'pre.txt','tag.txt

def SEC_data(dataset,quarter,year,filename):
    url = f"https://www.sec.gov/files/dera/data/{dataset}/{year}{quarter}.zip"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    df = pd.read_csv(filename,sep="\t",dtype={"cik":str})
    return df

In [5]:
financials = []    
for year in years:
    for quarter in quarters:
        for filename in filenames:
            df = SEC_data(dataset,quarter, year, filename)
            financials.append(df)

/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
numi = []
subi = []
financials_all = []

for n in list(range(len(financials))):
    if n % 2 == 0:
        numi.append(n)
    else:  
        subi.append(n)
        
for n, s in zip(numi,subi):
    merged_df = financials[n].merge(financials[s][["adsh","cik","name","period","fy","fp","filed", "form","instance"]])
    financials_all.append(merged_df)

Filter for only 10-Ks only
<br> Companies have different FYs and larger companies take much longer to submit filings -- will not filter for quarter 

In [7]:
financials = pd.concat(financials_all)
financials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105850078 entries, 0 to 2373223
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   tag       object 
 1   version   object 
 2   coreg     object 
 3   ddate     object 
 4   qtrs      object 
 5   uom       object 
 6   value     float64
 7   footnote  object 
 8   adsh      object 
 9   cik       object 
 10  name      object 
 11  period    float64
 12  fy        float64
 13  fp        object 
 14  filed     object 
 15  form      object 
 16  instance  object 
dtypes: float64(3), object(14)
memory usage: 14.2+ GB


In [17]:
tags = pd.DataFrame(financials['tag'].value_counts())
tags[tags['tag']>=500000]

,tag
NetIncomeLoss,1219874
CashAndCashEquivalentsAtCarryingValue,1090750
OperatingIncomeLoss,905307
IncomeTaxExpenseBenefit,853246
StockholdersEquity,795342
Assets,698393
InterestExpense,673430
LiabilitiesAndStockholdersEquity,665961
EarningsPerShareBasic,659969
EarningsPerShareDiluted,639670


In [18]:
financials_tags = financials[financials["tag"].isin(["NetIncomeLoss",
                                                 "CashAndCashEquivalentsAtCarryingValue",
                                                 "OperatingIncomeLoss",
                                                 "StockholdersEquity",
                                                 "Assets",
                                                 
                                                 "LiabilitiesAndStockholdersEquity",
                                                 "LiabilitiesAndStockholdersEquity",
                                                 
                                                 "EarningsPerShareBasic",
                                                 "EarningsPerShareDiluted",
                                                 
                                                 "NetCashProvidedByUsedInOperatingActivities",
                                                 "NetCashProvidedByUsedInFinancingActivities",
                                                 
                                                 "RetainedEarningsAccumulatedDeficit",
                                                 "EarningsPerShareDiluted",
                                                 "CommonStockSharesAuthorized"
                                                 ])]

In [19]:
annual_raw = financials_tags[financials_tags["form"].isin(["10-K","20-K","40-K"])]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2393323 entries, 3481 to 2288717
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   tag       object 
 1   version   object 
 2   coreg     object 
 3   ddate     object 
 4   qtrs      object 
 5   uom       object 
 6   value     float64
 7   footnote  object 
 8   adsh      object 
 9   cik       object 
 10  name      object 
 11  period    float64
 12  fy        float64
 13  fp        object 
 14  filed     object 
 15  form      object 
 16  instance  object 
dtypes: float64(3), object(14)
memory usage: 328.7+ MB


In [29]:
annual_raw.drop(annual_raw.columns[np.r_[7]], axis=1, inplace=True)


/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
annual_raw.drop_duplicates(inplace= True)

<ipython-input-30-f35c028b3ecf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annual_raw.drop_duplicates(inplace= True)


In [37]:
annual_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2393312 entries, 3481 to 2288717
Data columns (total 16 columns):
 #   Column    Dtype  
---  ------    -----  
 0   tag       object 
 1   version   object 
 2   coreg     object 
 3   ddate     object 
 4   qtrs      object 
 5   uom       object 
 6   value     float64
 7   adsh      object 
 8   cik       object 
 9   name      object 
 10  period    float64
 11  fy        float64
 12  fp        object 
 13  filed     object 
 14  form      object 
 15  instance  object 
dtypes: float64(3), object(13)
memory usage: 310.4+ MB


In [41]:
annual = annual_raw.pivot_table(values='value', 
                          index= ['version',
                                  'coreg',
                                  'ddate',
                                  'qtrs',
                                  'uom',
                                  'adsh',
                                  'cik',
                                  'name',
                                  'period',
                                  'fy',
                                  'fp',
                                  'filed',
                                  'form',
                                  'instance'],
                          columns = 'tag')


In [42]:
annual.reset_index(inplace=True)

In [46]:
annual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114475 entries, 0 to 114474
Data columns (total 26 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   version                                     114475 non-null  object 
 1   coreg                                       114475 non-null  object 
 2   ddate                                       114475 non-null  int64  
 3   qtrs                                        114475 non-null  int64  
 4   uom                                         114475 non-null  object 
 5   adsh                                        114475 non-null  object 
 6   cik                                         114475 non-null  object 
 7   name                                        114475 non-null  object 
 8   period                                      114475 non-null  float64
 9   fy                                          114475 non-null  float64
 

# Crowdfunding Data

In [47]:
# import crowdfunding -- 'FORM_C_SUBMISSION.tsv','FORM_C_DISCLOSURE.tsv'
quarters = ['q1_cf','q2_cf','q3_cf','q4_cf']
years = list(range(2017,2021))
dataset = 'crowdfunding-offerings-data-sets'

crowdfunding = []    
for year in years:
    for quarter in quarters:
        file_disc = f"{year}{quarter.capitalize()}/FORM_C_DISCLOSURE.tsv"
        file_issuer = f"{year}{quarter.capitalize()}/FORM_C_ISSUER_INFORMATION.tsv"        
        filenames =[file_disc,file_issuer]
        for filename in filenames:
            df = SEC_data(dataset, quarter, year, filename)
            crowdfunding.append(df)
 

In [48]:
disc_i = []
issuer_i = []
crowdfunding_all = []

for n in list(range(len(crowdfunding))):
    if n % 2 == 0:
        disc_i.append(n)
    else:  
        issuer_i.append(n)
        
for n, s in zip(disc_i,issuer_i):
    df = crowdfunding[n].merge(crowdfunding[s])
    crowdfunding_all.append(df)

In [54]:
crowdfunding = pd.concat(crowdfunding_all)

In [55]:
crowdfunding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9553 entries, 0 to 1013
Data columns (total 51 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ACCESSION_NUMBER                9553 non-null   object 
 1   COMPENSATIONAMOUNT              7958 non-null   object 
 2   FINANCIALINTEREST               5716 non-null   object 
 3   SECURITYOFFEREDTYPE             7958 non-null   object 
 4   SECURITYOFFEREDOTHERDESC        3625 non-null   object 
 5   NOOFSECURITYOFFERED             6401 non-null   float64
 6   PRICE                           7300 non-null   float64
 7   PRICEDETERMINATIONMETHOD        4245 non-null   object 
 8   OFFERINGAMOUNT                  7958 non-null   float64
 9   OVERSUBSCRIPTIONACCEPTED        7958 non-null   object 
 10  OVERSUBSCRIPTIONALLOCATIONTYPE  7875 non-null   object 
 11  DESCOVERSUBSCRIPTION            4575 non-null   object 
 12  MAXIMUMOFFERINGAMOUNT           78

In [61]:
crowdfunding = crowdfunding[["COMPENSATIONAMOUNT",
                                         "SECURITYOFFEREDTYPE",
                                         "NOOFSECURITYOFFERED",
                                         "PRICE",                 
                                         "CURRENTEMPLOYEES",
                                         "NAMEOFISSUER",
                                         "LEGALSTATUSFORM",
                                         "DATEINCORPORATION",
                                         "JURISDICTIONORGANIZATION",
                                         "COMPANYNAME",
                                         "ZIPCODE"]]
                                         
           

In [62]:
crowdfunding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9553 entries, 0 to 1013
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   COMPENSATIONAMOUNT        7958 non-null   object 
 1   SECURITYOFFEREDTYPE       7958 non-null   object 
 2   NOOFSECURITYOFFERED       6401 non-null   float64
 3   PRICE                     7300 non-null   float64
 4   CURRENTEMPLOYEES          9045 non-null   float64
 5   NAMEOFISSUER              9553 non-null   object 
 6   LEGALSTATUSFORM           9233 non-null   object 
 7   DATEINCORPORATION         9233 non-null   object 
 8   JURISDICTIONORGANIZATION  9233 non-null   object 
 9   COMPANYNAME               7980 non-null   object 
 10  ZIPCODE                   9233 non-null   object 
dtypes: float64(3), object(8)
memory usage: 895.6+ KB
